In [3]:
with open('dana.filtered.out', 'r') as file:
    gL = []
    for line in file:
        line = line.strip().split('\t')
        gL.append(line[0])
sing_count = 0
sing_list = []
for x in set(gL):
    if gL.count(x) == 1:
        sing_count += 1
        sing_list.append(x)
print(sing_count)

2917


In [6]:
import requests, json, time
from ete3 import PhyloTree
singTreeList, singTreeListPruned, count, ti = [], [], 0, time.time()
noTree = 0
print('Fetching gene trees...')
for g in sing_list:
    url = 'http://rest.ensemblgenomes.org/genetree/member/id/'+ g + '?compara=metazoa;nh_format=display_label_composite;object_type=Translation;'
    headers = {'Content-Type':'text/x-nh'}
    tree = requests.get(url, headers=headers).text
    if "error" in tree:
        noTree += 1
    else:
        singTreeList.append(tree)
    count += 1
    if count % 200 == 0:
        print(count, 'done', round(time.time()-ti, 2), 'seconds')
print('Pruning...')
count, ti = 0, time.time()
for i in range(0,len(singTreeList)):
    try:
        t = PhyloTree(singTreeList[i])
    except:
        print('tree making failed')
        print(singTreeList[i])
    pList = []
    nodes = ['Dere', 'Dyak', 'Dsec', 'Dsim', 'Dmel', 'Dana', 'Dpse']
    for n in nodes:
        for node in t.traverse():
            if n in node.name:
                 pList.append(node.name)
    try:
        t.prune(pList, preserve_branch_length=True)
    except:
        print('pruning failed')
        print(t)
    singTreeListPruned.append(t)
    count += 1
    if count % 200 == 0:
        print(count, 'done', round(time.time()-ti,2), 'seconds')

Fetching gene trees...
200 done 25.82 seconds
400 done 50.42 seconds
600 done 74.88 seconds
800 done 98.98 seconds
1000 done 124.02 seconds
1200 done 149.35 seconds
1400 done 172.94 seconds
1600 done 197.83 seconds
1800 done 221.55 seconds
2000 done 244.55 seconds
2200 done 269.42 seconds
2400 done 295.24 seconds
2600 done 322.95 seconds
2800 done 351.09 seconds
Pruning...
200 done 3.5 seconds
400 done 7.34 seconds
600 done 10.32 seconds
800 done 13.31 seconds
1000 done 15.52 seconds
1200 done 18.91 seconds
1400 done 22.24 seconds
1600 done 25.41 seconds
1800 done 28.13 seconds
2000 done 30.91 seconds
2200 done 33.44 seconds
2400 done 37.94 seconds
2600 done 41.5 seconds


In [11]:
singTreeListPrunedPseudo, noPseudo = [], 0
for tree in singTreeListPruned:
    pseudo = False
    for node in tree.traverse():
        if 'Dpse' in node.name:
            pseudo = True
    if pseudo == True:
        singTreeListPrunedPseudo.append(tree)
    else:
        noPseudo += 1
print(noPseudo)

251


In [12]:
singTreeOneEach, singTreeMulti = [], []
for tree in singTreeListPrunedPseudo:
    Pseudo, Ana = 0,0
    for node in tree.traverse():
        if 'Dpse' in node.name:
            Pseudo += 1
        if 'Dana' in node.name:
            Ana += 1
    if Ana == 1 and Pseudo == 1:
        singTreeOneEach.append(tree)
    else:
        singTreeMulti.append(tree)

In [49]:
top_dict = {}
for i in range(0, len(singTreeMulti)):
    top = singTreeMulti[i].get_topology_id(attr='species')
    try:
        top_dict[top].append(i)
    except KeyError:
        top_dict[top] = [i]

In [45]:
def get_species(name):
    return(name[-4:])
for tree in singTreeMulti:
    
    for n in tree.traverse():
        n.set_species_naming_function(get_species)

#     sp = n.name[-4:]
#     n.add_feature('species', sp)

In [40]:
dir(test)

['_PhyloNode__get_speciation_trees_recursive',
 '__add__',
 '__and__',
 '__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_asciiArt',
 '_children',
 '_diff',
 '_dist',
 '_get_children',
 '_get_dist',
 '_get_face_areas',
 '_get_farthest_and_closest_leaves',
 '_get_species',
 '_get_style',
 '_get_support',
 '_get_up',
 '_img_style',
 '_iter_descendants_levelorder',
 '_iter_descendants_postorder',
 '_iter_descendants_preorder',
 '_name',
 '_set_children',
 '_set_dist',
 '_set_face_areas',
 '_set_species',
 '_set_style',
 '_set_support',
 '_set_up',
 '_species',
 '_speciesFunction',
 '_support',
 '_up',
 'add_child',
 'add_fac

In [51]:
len(singTreeMulti)

152

In [53]:
id_list = []
for key in top_dict:
    id_list.append(key)

In [201]:
singTreeMulti[top_dict[id_list[70]][0]].show()

IndexError: list index out of range

In [57]:
res_list = []

In [200]:
res_list.append('Weird')

In [211]:
maybe_list = []
for top_id in id_list:
    if res_list[id_list.index(top_id)] == 'Yes' or res_list[id_list.index(top_id)] == 'Weird':
        maybe_list.append(top_id)

In [213]:
len(maybe_list)

19

In [215]:
ex_list = []
for top_id in id_list:
    if res_list[id_list.index(top_id)] == 'No':
        ex_list.append(top_id)
with open('excluded_multi_topologies.txt', 'w') as file:
    for x in ex_list:
        file.write(x + '\n')

In [217]:
singTreeMultiMaybe = []
for tree in singTreeMulti:
    if tree.get_topology_id(attr='species') in maybe_list:
        singTreeMultiMaybe.append(tree)

In [218]:
for tree in singTreeOneEach:
    
    for n in tree.traverse():
        n.set_species_naming_function(get_species)

In [219]:
top_dict_one_each = {}
for i in range(0, len(singTreeOneEach)):
    top = singTreeOneEach[i].get_topology_id(attr='species')
    try:
        top_dict_one_each[top].append(i)
    except KeyError:
        top_dict_one_each[top] = [i]
print(len(top_dict_one_each))

179


In [262]:
singTreeMultiMaybe[7].show()

In [227]:
finalTreeList = []

In [333]:
# manually looking at trees to divide up
tree = singTreeMultiMaybe[15] #15 was the last index done
node = tree.search_nodes(name = "DpseGA30490_FBpp0308192_Dpse")[0]
det = node.get_common_ancestor(tree.search_nodes(name = "DereGG19859_FBpp0138405_Dere")[0])
det.detach()
print(det)


            /-DyakGE11385_FBpp0256395_Dyak
         /-|
        |   \-DereGG19859_FBpp0138405_Dere
      /-|
     |  |      /-DsecGM11763_FBpp0193240_Dsec
     |  |   /-|
   /-|   \-|   \-DsimGD24896_FBpp0223298_Dsim
  |  |     |
  |  |      \-lov_FBpp0303264_Dmel
--|  |
  |   \-DanaGF11509_FBpp0347040_Dana
  |
   \-DpseGA30490_FBpp0308192_Dpse


In [334]:
finalTreeList.append(det)

In [299]:
notTrustedList.append(singTreeMultiMaybe[14])

In [352]:
print(singTreeMultiMaybe[15])


                                 /-DsecGM20503_FBpp0201980_Dsec
                              /-|
                           /-|   \-Dsimlola_FBpp0326119_Dsim
                          |  |
                        /-|   \-lola_FBpp0309831_Dmel
                       |  |
                     /-|   \-DyakGE13086_FBpp0364001_Dyak
                    |  |
                  /-|   \-DereGG22728_FBpp0367010_Dere
                 |  |
               /-|   \-DanaGF11096_FBpp0348111_Dana
              |  |
         /- /-|   \-DpseGA30442_FBpp0335721_Dpse
        |     |
        |     |   /-DpseGA31686_FBpp0330359_Dpse
      /-|      \-|
     |  |         \-DpseGA25769_FBpp0333863_Dpse
     |  |
     |   \-DsecGM11014_FBpp0192491_Dsec
     |
     |                           /-DsimGD16767_FBpp0313503_Dsim
     |                        /-|
     |                     /-|   \-DsecGM12450_FBpp0193927_Dsec
     |                    |  |
     |                  /-|   \-CG3726_FBpp0309797_Dmel
     |  

In [362]:
for x in finalTreeList:
    test = x
    finalTreeList.index(x)
    p_list, a_list = [],[]
    for n in test.traverse():
        if n.species == 'Dpse':
            p_list.append(n)
        elif n.species == 'Dana':
            a_list.append(n)
    for n1 in p_list:
        for n2 in a_list:
            print(n1.name, n2.name, test.get_distance(n1,n2))

DpseGA23934_FBpp0329174_Dpse DanaGF24852_FBpp0343615_Dana 1.192373
DpseGA20928_FBpp0334330_Dpse DanaGF27615_FBpp0348167_Dana 0.6386449999999999
DpseGA10760_FBpp0341065_Dpse DanaGF12705_FBpp0346424_Dana 0.309903
DpseGA33017_FBpp0329599_Dpse DanaGF28152_FBpp0343745_Dana 0.95478
DpseGA13817_FBpp0332868_Dpse DanaGF17203_FBpp0120395_Dana 0.473858
DpseGA10575_FBpp0329069_Dpse DanaGF19009_FBpp0350007_Dana 0.256688
DpseGA26552_FBpp0329471_Dpse DanaGF18734_FBpp0343043_Dana 0.277708
DpseGA24988_FBpp0336309_Dpse DanaGF12927_FBpp0346064_Dana 0.15546
DpseGA13817_FBpp0332868_Dpse DanaGF17203_FBpp0120395_Dana 0.473858
DpseGA19085_FBpp0334840_Dpse DanaGF10336_FBpp0113528_Dana 0.27106800000000003
DpseGA19161_FBpp0280114_Dpse DanaGF14376_FBpp0117568_Dana 0.606086
DpseGA18777_FBpp0335092_Dpse DanaGF21941_FBpp0125133_Dana 0.46107200000000004
DpseGA16409_FBpp0280130_Dpse DanaGF15341_FBpp0118533_Dana 0.444771
DpseGA16410_FBpp0280129_Dpse DanaGF15340_FBpp0118532_Dana 0.553123
DpseGA10454_FBpp0335143_Dpse Dan

In [365]:
finalTreeList[7].show()

In [358]:
print(finalTreeList[2])


                  /-DsimGD10469_FBpp0327434_Dsim
               /-|
            /-|   \-DsecGM20944_FBpp0202421_Dsec
           |  |
         /-|   \-pwn_FBpp0304734_Dmel
        |  |
      /-|   \-DyakGE19119_FBpp0352845_Dyak
     |  |
   /-|   \-DereGG23271_FBpp0366361_Dere
  |  |
--|   \-DanaGF12705_FBpp0346424_Dana
  |
   \-DpseGA10760_FBpp0341065_Dpse
